In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")

True

In [2]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")

In [7]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
tools = load_tools(["llm-math"], llm=llm)

In [8]:
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish

class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
        try:
            # this will work IF the text is a valid JSON with action and action_input
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                # this means the agent is finished so we call AgentFinish
                return AgentFinish({"output": action_input}, text)
            else:
                # otherwise the agent wants to use an action, so we call AgentAction
                return AgentAction(action, action_input, text)
        except Exception:
            # sometimes the agent will return a string that is not a valid JSON
            # often this happens when the agent is finished
            # so we just return the text as the output
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

# initialize output parser for agent
parser = OutputParser()

In [9]:
from langchain.agents import initialize_agent

# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    #agent_kwargs={"output_parser": parser}
)

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and 

In [11]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

In [12]:
sys_msg = B_SYS + """Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {{"action": "Calculator",
      "action_input": "sqrt(4)"}}
    ```

Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}}
```
User: I'm great, what is the square root of 4?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "sqrt(4)"}}
```
User: 2.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 2!"}}
```
User: Thanks could you tell me what 4 to the power of 2 is?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "4**2"}}
```
User: 16.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 16!"}}
```

Here is the latest conversation between Assistant and User.""" + E_SYS
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
agent.agent.llm_chain.prompt = new_prompt

In [13]:
instruction = B_INST + " Respond to the following in JSON with 'action' and 'action_input' values " + E_INST
human_msg = instruction + "\nUser: {input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

In [14]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<>\nAssistant is a expert JSON builder designed to assist with a wide range of tasks.\n\nAssistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_inp

In [15]:
agent("hey how are you today?")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{"action": "Final Answer",
 "action_input": "I'm well, how are you?"}
```

> Finished chain.


{'input': 'hey how are you today?',
 'chat_history': [],
 'output': "I'm well, how are you?"}

In [16]:
agent("what is 4 to the power of 2.1?")



> Entering new AgentExecutor chain...
```json
{
  "action": "Calculator",
  "action_input": "4**2.1"
}
```
Observation: Answer: 18.37917367995256
Thought:```json
{
  "action": "Final Answer",
  "action_input": "The answer is 18.37917367995256!"
}
```

> Finished chain.


{'input': 'what is 4 to the power of 2.1?',
 'chat_history': [HumanMessage(content='hey how are you today?'),
  AIMessage(content="I'm well, how are you?")],
 'output': 'The answer is 18.37917367995256!'}

In [17]:
agent("can you multiply that by 3?")



> Entering new AgentExecutor chain...
```json
{"action": "Calculator",
 "action_input": "18.37917367995256*3"}
```
Observation: Answer: 55.13752103985769
Thought:```json
{"action": "Final Answer",
 "action_input": "The answer is 55.13752103985769!"}
```

> Finished chain.


{'input': 'can you multiply that by 3?',
 'chat_history': [HumanMessage(content='hey how are you today?'),
  AIMessage(content="I'm well, how are you?"),
  HumanMessage(content='what is 4 to the power of 2.1?'),
  AIMessage(content='The answer is 18.37917367995256!')],
 'output': 'The answer is 55.13752103985769!'}

In [18]:
agent("can you devide that by 5?")



> Entering new AgentExecutor chain...
```json
{
  "action": "Calculator",
  "action_input": "55.13752103985769/5"
}
```
Observation: Answer: 11.027504207971537
Thought:```json
{
  "action": "Final Answer",
  "action_input": "The answer is 11.027504207971537!"
}
```

> Finished chain.


{'input': 'can you devide that by 5?',
 'chat_history': [HumanMessage(content='hey how are you today?'),
  AIMessage(content="I'm well, how are you?"),
  HumanMessage(content='what is 4 to the power of 2.1?'),
  AIMessage(content='The answer is 18.37917367995256!'),
  HumanMessage(content='can you multiply that by 3?'),
  AIMessage(content='The answer is 55.13752103985769!')],
 'output': 'The answer is 11.027504207971537!'}

In [22]:
agent("round of that?")



> Entering new AgentExecutor chain...
```json
{"action": "Calculator",
 "action_input": "round(11.027504207971537)"}
```

ValueError: LLMMathChain._evaluate("
round(11.027504207971537)
") raised error: 'VariableNode' object is not callable. Please try again with a valid numerical expression